<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/medcat_normalisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Mount Google Drive
#drive.flush_and_unmount()
#!rm -rf /content/drive
#!mkdir -p /content/drive

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [3]:
!pip install -q medcat
!python -m spacy download en_core_web_md
from medcat.cat import CAT
import os
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.8/307.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 53.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [6]:
# Load the model pack - almost 1GB so takes a while

medcat_zipfile_location = "/content/drive/My Drive/Colab Notebooks/Dissertation/MedCAT"

os.makedirs(medcat_zipfile_location, exist_ok=True)

medcat_model_zipfile = 'v2_Snomed2025_MIMIC_IV_bbe806e192df009f.zip'

MEDCAT_LOAD = os.path.join(medcat_zipfile_location, medcat_model_zipfile)

cat = CAT.load_model_pack(MEDCAT_LOAD)

In [7]:
# Path variables

LOAD_PATH = "/content/drive/My Drive/Colab Notebooks/Dissertation/JSON"
LOAD_GUIDELINE_STRUCTURED = os.path.join(LOAD_PATH, "guideline_structured.json")

In [8]:
# Load JSON

def load_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f'JSON file not found: {file_path}')

guideline_structured = load_json(LOAD_GUIDELINE_STRUCTURED)

print(type(guideline_structured))
print(len(guideline_structured))
print(guideline_structured)

<class 'list'>
116
[{'heading_context': '1.1 Care for adults, children and young people across all phases of bipolar disorder > Treatment and support for specific populations', 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.', 'conditionality': None, 'action': ['offer the same range of treatments and services'], 'information_giving': None, 'clinical_setting': None, 'population': 'older people with bipolar disorder', 'age_group': 'older_adult', 'prohibitions_and_cautions': None, 'urgency': False, 'manic_episode_history': None, 'current_manic_phase': None, 'symptom_severity': None, 'current_psychosis': None, 'diagnoses': ['bipolar disorder'], 'current_medication': None, 'medication_adherence': None, 'suggested_medication': None, 'physical_health_longterm': None, 'physical_health_recent': None, 'risk': None, 'psychologica

In [10]:
# Test (from original documentation)

text = "The patient has a history of ectopic pregnancy but no current symptoms of miscarriage."
entities = cat.get_entities(text)

# This returns a dictionary of entities found

for entity_id, entity_data in entities['entities'].items():
    print(f"Term: {entity_data['source_value']}")
    print(f"SNOMED/CUI: {entity_data['cui']}")
    print(f"Confidence: {entity_data['context_similarity']}")

    # If using meta-annotations (Status/Certainty):
    print(f"Meta-Annotations: {entity_data.get('meta_anns')}")

Term: patient has
SNOMED/CUI: 25609006
Confidence: 1
Meta-Annotations: {}
Term: history of ectopic pregnancy
SNOMED/CUI: 161763005
Confidence: 1
Meta-Annotations: {}
Term: current
SNOMED/CUI: 15240007
Confidence: 1
Meta-Annotations: {}
Term: miscarriage
SNOMED/CUI: 17369002
Confidence: 1
Meta-Annotations: {}


In [12]:
def medcat_norm(structured_json_file):

    for entry in structured_json_file: # Because the outer structure is a list []

        for key, value in list(entry.items()): # While the inner structure is a dictionary {} so uses .items()
            if key == 'diagnoses':
                joined_text = " ".join(value) if isinstance(value, list) else (value or "")
                entry['diagnosis_normed'] = cat.get_entities(joined_text)

            elif key == 'current_medication':
                joined_text = " ".join(value) if isinstance(value, list) else (value or "")
                entry['current_medication_normed'] = cat.get_entities(joined_text)

            elif key == 'physical_health_longterm':
                joined_text = " ".join(value) if isinstance(value, list) else (value or "")
                entry['physical_health_longterm_normed'] = cat.get_entities(joined_text)

            elif key == 'physical_health_recent':
                joined_text = " ".join(value) if isinstance(value, list) else (value or "")
                entry['physical_health_recent_normed'] = cat.get_entities(joined_text)

    with open('/content/drive/My Drive/Colab Notebooks/Dissertation/JSON/guideline_structured_normed.json', "w", encoding="utf-8") as f:
        json.dump(structured_json_file, f, ensure_ascii=False, indent=2)

    return structured_json_file

In [14]:
data = medcat_norm(guideline_structured)

print(data)

[{'heading_context': '1.1 Care for adults, children and young people across all phases of bipolar disorder > Treatment and support for specific populations', 'original_recommendation_number': '1.1.1', 'original_recommendation_text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder.', 'conditionality': None, 'action': ['offer the same range of treatments and services'], 'information_giving': None, 'clinical_setting': None, 'population': 'older people with bipolar disorder', 'age_group': 'older_adult', 'prohibitions_and_cautions': None, 'urgency': False, 'manic_episode_history': None, 'current_manic_phase': None, 'symptom_severity': None, 'current_psychosis': None, 'diagnoses': ['bipolar disorder'], 'current_medication': None, 'medication_adherence': None, 'suggested_medication': None, 'physical_health_longterm': None, 'physical_health_recent': None, 'risk': None, 'psychological_therapy': False, 